# Table agrégée commune année : exemple des nitrites

## 1. Introduction et récupération des données brutes

Ce Notebook vise à compléter la partie nitrite de la tâche 167 
[définie ici](https://noco.services.dataforgood.fr/dashboard/#/nc/pk1vq1pm8frc5lm/ms9uz8er4jpow7j/Tasks%20data%20analyst?rowId=167)


Ce faisant, nous définissions plusieurs fonctions d'appoint orientées Python/Pandas pour ingérer et traiter les bases de données afin de récupérer aisément les colonnes de polluants correspondant à la catégorie désirée.


On commence par récupérer les tables.

On utilise la BDD locale pour les prélèvements.

Pour l'appariement des catégories, la Google sheet utilisée (à croiser avec l'autre de GF) sera  :
https://docs.google.com/spreadsheets/d/1W239EkEV72WzzefB6jbYUQkIIQ-9xa4u9_m8XdCpU4o/edit?gid=973653272#gid=973653272



In [ ]:
import pandas as pd
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE

pd.set_option("display.max_columns", None)  # show all cols
con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

results = con.table("edc_resultats")

con.execute("SHOW TABLES").df()

In [ ]:
con.table("edc_communes").show()

In [ ]:
con.table("edc_prelevements").limit(5).df()

In [ ]:
def fetch_google_sheet():
    # Récupère la google sheet, adapté de :
    # https://medium.com/@Bwhiz/step-by-step-guide-importing-google-sheets-data-into-pandas-ae2df899257f
    sheet_id = (
        "1W239EkEV72WzzefB6jbYUQkIIQ-9xa4u9_m8XdCpU4o"  # replace with your sheet's ID
    )
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv"
    return pd.read_csv(url)


def get_map_cat(cle: str = "cdparametresiseeaux"):
    # Ajoute à la gsheet les seuils récupérés de limitequal dans edc_resultats
    # On fait un merge assez sauvage qui ne garde que les cas qui se passent bien.
    dfgsheet = fetch_google_sheet()
    dfseuil = results.select(cle, "limitequal").distinct().df().dropna()
    dfseuil["limit"] = (
        dfseuil["limitequal"]
        .str.replace(",", ".")
        .str.extract(r"([\d.]+)")
        .astype(float)
    )
    return dfgsheet.merge(dfseuil[[cle, "limit"]])


dfmapcat = get_map_cat()
dfmapcat.groupby("categorie").first()

## 2. Fonctions de pivot

Les fonctions suivantes permettent d'obtenir la table correspondant aux catégories souhaitées.
* pivot_resultats_pour_categories fait pivoter la table des résultats en filtrant pour ne garder que les colonnes précisées dans categories
* calcul_depassements ajoute les colonnes précisant si les seuils sont dépassés en utilisant les seuils de edc_resultats (pas utilisé ici)
* fusion_prel_pivot_resultats fusionne avec la table edc_prelevements sur referenceprel en ajoutant les colonnes prel_col


In [ ]:
def pivot_resultats_pour_categories(
    categories: list,
    index: str = "referenceprel",
    columns: str = "cdparametresiseeaux",
    values: str = "valtraduite",
):
    rel = results.select(index, columns, values).filter(f"{columns} in {categories}")
    dfpiv = (
        rel.df()
        .drop_duplicates(subset=[index, columns])
        .pivot(index=index, columns=columns)
    )
    dfpiv.columns = [multilevel[1] for multilevel in dfpiv.columns]
    return dfpiv


def calcul_depassements(dseuil: dict):
    """dseuil: dict of categorie(str): seuil(float)
    return
    """
    dfpiv = pivot_resultats_pour_categories(list(dseuil.keys()))
    for col, seuil in dseuil.items():
        dfpiv["b_" + col] = dfpiv[col] > seuil
    return dfpiv


def fusion_prel_pivot_resultats(
    dfpiv, prel_col=["inseecommuneprinc", "nomcommuneprinc", "dateprel", "de_partition"]
):
    ref_col = "referenceprel"
    dfprel = (
        con.table("edc_prelevements")
        .select(ref_col, *prel_col)
        .df()
        .drop_duplicates(subset=ref_col)
        .set_index(ref_col)
    )
    dfmerge = dfprel.merge(
        dfpiv, left_index=True, right_index=True, validate="one_to_one"
    )
    return dfmerge

## 3. Application aux nitrites


Nous voulons maintenant avoir les résultats pour les nitrites.
Les détails de ce qu'on veut sont disponibles sur [ce Google Doc](https://docs.google.com/document/d/1foeVEWU6Kf12Fo-_Vj3vVwyE-3LXGzZPlofEYDCcstk/edit?tab=t.0)

Besoin de 

* Filtre temporel
    * (à choisir en premier)
    * Résultats des dernières analyses
    * OU Bilan annuel (2020 -> 2024)
    * pour le moment seul "résultat des dernières analyses" est décrit, on se concentre là-dessus.


* Filtres par catégories de polluants [Lien Google Sheets](https://docs.google.com/spreadsheets/d/1a5ywbeeu5_T8F3707wIC8WX86_0577wK/edit?usp=sharing&ouid=115470011005689616061&rtpof=true&sd=true )
    * pour rappel cf. première cellule on utilise un autre tableau Google Sheet, plus commode
    * L'affichage dépend du type de polluant sélectionné
    * Ce notebook : nitrites et nitrates


* Si une commune contient plusieurs UDI, le résultat pire cas est affiché (une seule UDI dépasse une norme, la commune passe en orange / rouge)
    -> pas traité ici


* Nitrites, 5 situations:
    * Nitrates < 25 mg/L et nitrites < 0,5 mg/L (eau conforme)
    * Nitrates entre 25 et 40 mg/L et nitrites < 0,5 mg/L (eau conforme)
    * Nitrates entre 40 et 50 mg/L et nitrites < 0,5 mg/L (eau conforme)
    * Nitrates entre 50 et 100 mg/L et/ou nitrites > 0,5 mg/L (eau non conforme) (fait passer l’affichage total polluant en rouge)
    * Nitrates > 100 mg/L et/ou nitrites > 0,5 mg/L (eau non conforme) (fait passer l’affichage total polluant en rouge)


In [ ]:
dfnitrite = dfmapcat.query("categorie=='nitrite'")
dfnitrite

In [ ]:
dfpiv = pivot_resultats_pour_categories(["NO3", "NO2", "NO3_NO2"])
dffus = fusion_prel_pivot_resultats(dfpiv)
dffus

In [ ]:
# Y a-t-il des cas où on n'a pas NO2 mais NO3_NO2 ?
dffus[(dffus["NO2"].isna()) & (~dffus["NO3_NO2"].isna())]

In [ ]:
# Y a-t-il des cas où on n'a pas NO3 mais NO3_NO2 ?
dffus[(dffus["NO3"].isna()) & (~dffus["NO3_NO2"].isna())]

In [ ]:
# Y a-t-il des cas où on n'a ni NO2 ni NO3 mais NO3_NO2 ? (tant pis pour eux...)
dffus[(dffus["NO2"].isna()) & (dffus["NO3"].isna()) & (~dffus["NO3_NO2"].isna())]

In [ ]:
# On regarde les distributions des paramètres d'intérêt

import numpy as np

dffus[["NO3", "NO2"]].hist(bins=np.linspace(0, 100, 100), log=True)

### Calcul de l'arbre de décision

On définit une catégorie pour chaque cas :
* 0:    Pas de données pour NO2 et NO3 (si un seul disponible on consière que l'autre est à 0 (!!!)
* 1:    Nitrates < 25 mg/L et nitrites < 0,5 mg/L (eau conforme)
* 2:    Nitrates entre 25 et 40 mg/L et nitrites < 0,5 mg/L (eau conforme)
* 3:    Nitrates entre 40 et 50 mg/L et nitrites < 0,5 mg/L (eau conforme)
* 4:    Nitrates entre 50 et 100 mg/L et/ou nitrites > 0,5 mg/L (eau non conforme) (fait passer l’affichage total polluant en rouge)
* 5:    Nitrates > 100 mg/L et/ou nitrites > 0,5 mg/L (eau non conforme) (fait passer l’affichage total polluant en rouge)


In [ ]:
def complete_nitrite_nitrate(row):
    NO2 = row["NO2"]
    NO3 = row["NO3"]
    NO3_NO2 = row["NO3_NO2"]  # Nitrates/50 + Nitrites/3 	= NO3/50 + N02/3
    if np.isnan(NO2) and np.isnan(NO2):
        return None, None
    if np.isnan(NO2):
        NO2 = 0 if np.isnan(NO3_NO2) else 3 * (NO3_NO2 - NO3 / 50)
    if np.isnan(NO3):
        NO3 = 0 if np.isnan(NO3_NO2) else 50 * (NO3_NO2 - NO2 / 3)
    return NO2, NO3


def arbre_decision(row):
    nitrite, nitrate = complete_nitrite_nitrate(row)
    if nitrate is None and nitrite is None:
        return 0
    elif nitrate < 25 and nitrite < 0.5:
        return 1
    elif nitrate < 40 and nitrite < 0.5:
        return 2
    elif nitrate < 50 and nitrite < 0.5:
        return 3
    elif nitrate > 100 or nitrite >= 0.5:
        return 5  #!
    else:
        return 4


dffus["resultat"] = dffus.apply(arbre_decision, axis=1)
dffus

## 4. Sanity check et visualisation des résultats

On garde uniquement les prélévements qui ont des résultats > 0, et la date du dernier prélèvement.

On affiche selon un code couleur qui va bien.

In [ ]:
dfres = (
    dffus.query("resultat>0")
    .sort_values(["dateprel", "inseecommuneprinc"])
    .groupby("inseecommuneprinc", as_index=False)
    .last()
)
dfres.resultat.value_counts()

In [ ]:
!uv pip install geopandas shapely folium matplotlib mapclassify seaborn

In [ ]:
import seaborn as sns

pal = sns.color_palette("hls", 10)
pal

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

cmap = pal.as_hex()[:5][
    ::-1
]  # On garde les 5 premières couleurs, inversées pour aller du vert au rouge
dfgeo = (
    con.table("laposte_communes")
    .df()[["code_commune_insee", "_geopoint"]]
    .set_index("code_commune_insee")
    .dropna()
)
dgeo = {k: Point(eval(v["_geopoint"])[::-1]) for k, v in dfgeo.iterrows()}
dfg = gpd.GeoDataFrame(
    data=dfres, geometry=dfres["inseecommuneprinc"].map(dgeo), crs="WGS84"
)
dfg

In [ ]:
dfg.explore(column="resultat", categorical=True, cmap=cmap, marker_type="circle")